In [1]:
# Los imports
import os
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

import networkx as nx

# Red de Colaboraciones de "Agrupaciones Gaiteras"

Me encanta el género musical __"Gaita Zuliana"__, es una expresión del folklore de mi ciudad natal, Maracaibo, Venezuela, así que como primer regalo de mi parte para mi ciudad natal y los "gaiteros", haré un análisis de red de colaboración de una base de datos reducida de Internet.

Para hacerlo, recopilaré los datos necesarios utilizando la poderosa combinación de las librerías de Python __Selenium__ y __Beautifulsoup__ para el "scraping" de datos web, pero esto solo se mostrará en este "notebook".

Luego, los datos obtenidos se analizarán utilizando __NetworkX__, una biblioteca de Python para el estudio de redes complejas, pero esto se hará en un "notebook" separado.

Finalmente, usaremos un modelo de lenguaje de Huggingface para extraer "entidades nombradas" __(NER, Named Entity Recognition)__ relacionadas con los artistas, específicamente sus nombres. A través de este proceso, nuestro objetivo es descubrir las relaciones y colaboraciones dentro de la hermosa escena musical de Gaita Zuliana.

## Scraping de los enlaces que contienen la información

La información será extraída de una página web llamada "Sabor Gaitero", y la forma en que se presenta la información es la siguiente:

* Hay una sección llamada "Agrupaciones Gaiteras", muy parecida a una página de resultados de búsqueda, donde se menciona cada grupo y tiene un hipervínculo que apunta a su información en forma de texto. Puedes ver la página en el siguiente enlace:

http://saborgaitero.com/agrupaciones-gaiteras/

* En cada página de un grupo "Gaitero", hay varios párrafos que contienen la información en forma de texto, y es exactamente esta información la que necesitamos extraer. A continuación un ejemplo individual de página de un grupo de gaitas:

http://saborgaitero.com/zagalines-los/

Como primer paso, vamos a crear una función de inicio de navegador que use __Selenuim__:

In [2]:
# HOME_DIR = os.path.expanduser('~')
DRIVER_FILE = r'C:\Users\armedina\Documents\DS-Projects\Deep_Gaitas\Drivers\chromedriver.exe'
# DRIVER_FILE = os.path.join()

def browser_starter(driver_file):
    """
    Función que inicializa el objeto Browser Driver.
    Retorna un objeto "webdriver.Chrome()" que utilizaré en las distintas actividades de scraping
    """
    ## Setup chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless") # Ensure GUI is off
    chrome_options.add_argument("--no-sandbox")

    # # Instancia el driver del navegador
    webdriver_service = Service(DRIVER_FILE)
    browser = webdriver.Chrome(service=webdriver_service, options=chrome_options)

    return browser

La siguiente parte es una función que toma un objeto __"sopa"__ de __BS4__ y extrae los enlaces para cada grupo "Gaita":

In [3]:
def get_all_pages_from_agrupaciones_sabor_gaitero(sopa):
    """
    Función que obtiene todos los enlaces de las agrupaciones gaiteras
    desde la página web especificada.

    * Params:
        * sopa : BS4 soup object.
        
    * Returns:
        Una lista con los enlaces de las agrupaciones gaiteras en formato "cadena de texto"
    """
    
    links = []
    output_links = []
    UMBRAL_LONGITUD_TITULO = 6
    # Itera por los distintos títulos (que son hipervinculos), y obtiene los enlaces.
    for div in sopa.find_all('h3'):
        if len((str(div.a.get('href'))).split(' ')) > UMBRAL_LONGITUD_TITULO:
            continue
        else:
            links.append(str(div.a.get('href')).lower())
    # Algunas palabras dentro de los elementos inspeccionados no son requeridos, por lo que
    # filtramos dichos enlaces.
    STOP_WORDS = ['ano', 'año', 'himno', 'escudo', 'de', 'la', 'puente', 'sobre', 'lago']
    for enlace in links:
        for stop_word in STOP_WORDS:
            if stop_word in enlace:
                continue
            else:
                output_links.append(enlace)
    return output_links

A continuación, recorremos cada enlace de la página de búsqueda para obtener todos los enlaces que apuntan a todos los grupos de "Gaita":

In [4]:
# ## Obtener los enlaces de la página de agrupaciones gaiteras
PAGINA_WEB_AGRUPACIONES = r'http://saborgaitero.com/agrupaciones-gaiteras/'
# # Abre la página
browser = browser_starter(DRIVER_FILE)
browser.get(PAGINA_WEB_AGRUPACIONES)
TIEMPO_ESPERA = 1.2
# # # Obtiene el html para la busqueda de liricas
html = browser.page_source
soup = BeautifulSoup(html,'html.parser')
time.sleep(TIEMPO_ESPERA)

# # Primeros enlaces
enlaces_agrupaciones = get_all_pages_from_agrupaciones_sabor_gaitero(soup)

# # Subsiguientes enlaces
BOTON_UNICO = '/html[1]/body[1]/div[5]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[3]/a[2]'


# Itera por las páginas de resultados, y extrae los enlaces de las liricas de las gaitas (aunque se cuelan algunos extra)
for _ in range(20):
    try:
        browser.find_element('xpath', BOTON_UNICO).click()
        html = browser.page_source
        soup = BeautifulSoup(html,'html.parser')
        enlaces_agrupaciones += get_all_pages_from_agrupaciones_sabor_gaitero(soup)
        time.sleep(TIEMPO_ESPERA)
    except Exception as e:
        print('Ha ocurrido un problema')
        print(e)
    finally:
        pass

browser.close()
browser.quit()

Como parte final de la fase de "scraping", recorremos ahora todas las páginas que contienen el texto real que describe al grupo "Gaita":

In [5]:
# A partir de los enlaces obtener las líricas
browser = browser_starter(DRIVER_FILE)
soups = []
for enlace in set(enlaces_agrupaciones):
    try:
        browser.get(enlace)

        # Obtiene el html para la busqueda de liricas
        html = browser.page_source
        soup = BeautifulSoup(html,'html.parser')
        soups.append(soup)
        time.sleep(TIEMPO_ESPERA)
    except Exception as e:
        print('Ha ocurrido un problema')
        print(e)

agrupaciones = []

for sopa in soups:
    agrupacion = {}
    titulo = sopa.find('h1').text.lower()
    agrupacion['TITULO'] = agrupacion.get('TITULO', titulo)
    texto = ' '.join(parrafo.text.lower() for parrafo in sopa.find_all('p'))
    agrupacion['TEXTO'] = agrupacion.get('TEXTO', texto)

    
    agrupaciones.append(agrupacion)

browser.close()

Guardamos los datos en un Dataframe de Pandas de la siguiente manera:

In [6]:
agrupaciones_df = pd.DataFrame.from_dict(agrupaciones)
agrupaciones_df = agrupaciones_df.loc[~agrupaciones_df['TITULO'].duplicated()]

In [7]:
agrupaciones_df

,TITULO,TEXTO
0,santanita,fundado 1964 gladys vera / raiza portillo / ot...
1,las 15 grandes gaitas de la temporada 2015,“la música se pone al servicio de cualquier pe...
2,"cardenales del exito, los",fundado 1962 ricardo aguirre / ricardo cepeda ...
3,"zagales, los",fundado 1977 daniel méndez / luis germán brice...
4,maragaita,fundado 1987 astolfo romero / carlos méndez / ...
5,barrio obrero,fundado 1955 alberto silva / carmencita silva ...
6,13 años de saborgaitero.com,"en el año 2003, león magno montiel publicó el ..."
7,gran coquivacoa,fundado 1968 neguito borjas / osías acosta / e...
8,vhg (venezuela habla gaiteando),fundado 1989 danelo badell / renato aguirre / ...
9,gaiteros de ziruma,fundado 2004 ender rojas / alí ojeda / daniela...


## Preparación y limpieza de datos

Para continuar, necesitamos algunos conocimientos de dominio para identificar si toda la información extraída es válida y proceder con la limpieza de datos.

La primera etapa de limpieza será simplemente filtrar todas las filas que contengan entradas que no sean grupos "Gaiteros", y para hacer esto, solo usaremos nuestro conocimiento de dominio y crearemos una lista que contenga los grupos que no sean gaiteros para usarla como filtro:

In [8]:
non_gaiteros = [
    'bachelet regresó a presidir la patria de jara. crónica semanal por @leonmagnom',
    '13 años de saborgaitero.com',
    'marisela árraga, una mujer ideal',
    'las 15 grandes gaitas de la temporada 2015',
    'argenis carruyo 69 años de vida y cantando',
    'banco occidental de descuento en gaitas',
    'somos',
    'san isidro',
    'kongaby',
              ]

agrupaciones_df = agrupaciones_df[~agrupaciones_df['TITULO'].isin(non_gaiteros)]

Y pasamos de 48 filas en el dataframe, a solo 39:

In [9]:
agrupaciones_df.shape

(39, 2)

Para el segundo paso, necesitamos extraer los nombres de los miembros del grupo "Gaitero" del texto obtenido en la fase anterior, y vamos a hacer esto usando __NER__ (Reconocimiento de entidad de nombre por sus siglas en inglés) que es una característica de __"Clasificación de tokens"__ de los modelos de lenguaje, que está capacitado para reconocer nombres de personas dentro del texto evaluado (entre muchas otras cosas, como por ejemplo, son capaces de reconocer nombres de ciudades o países), más específicamente usaremos la biblioteca __Flair__ para el procesamiento del lenguaje, que tiene una función llamada __" Sequence Tagger"__, y la combinaremos con su modelo de lenguaje __"ner-spanish-large"__ para NER, que se puede descargar desde el hub __Huggingface__ en:

https://huggingface.co/flair/ner-spanish-large

Para obtener un excelente y breve tutorial sobre cómo aprovechar el poder de estos modelos de lenguaje, puede consultar la página de tutoriales de Huggingface para LM.

In [10]:
from flair.data import Sentence
from flair.models import SequenceTagger

In [11]:
# load tagger
tagger = SequenceTagger.load("flair/ner-spanish-large")

C:\Users\armedina\Documents\DS-Projects\python_envs\gaitas_env\lib\site-packages\huggingface_hub\file_download.py:594: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2023-02-08 22:19:27,302 loading file C:\Users\armedina\.flair\models\ner-spanish-large\045ad6c7dc21e0eb85935dce0544eec65f8c63c58412154df4dee7ff5f11665b.d4d3456316d2951bc100d060bd63a690b33af6d273adffa1b90df32328ed3257
2023-02-08 22:19:43,556 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


A continuación, necesitamos definir una función que tome el texto y extraiga los nombres de los "Gaiteros", usando __Token Classification__ como se mencionó anteriormente (esta función fue tomada de la sección de ejemplo de la página web del modelo de lenguaje, en el hub __Huggingface__, que se muestra en la introducción de esta sección):

In [12]:
def ner_converver(texto):
    sentence = Sentence(texto)
    tagger.predict(sentence)
    entities = []
    for entity in sentence.get_spans('ner'):
        if entity.tag == 'PER':
            entities.append(entity.text)
            
    return ','.join(entities)

Luego creamos una nueva columna llamada __NER__ que contendrá una cadena con los nombres de los "Gaiteros" de cada grupo, separados por una coma:

In [13]:
# predict NER tags
agrupaciones_df['NER'] = agrupaciones_df['TEXTO'].apply(lambda texto: ner_converver(texto))

C:\Users\armedina\AppData\Local\Temp\ipykernel_11216\4042971319.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agrupaciones_df['NER'] = agrupaciones_df['TEXTO'].apply(lambda texto: ner_converver(texto))


In [14]:
agrupaciones_df

,TITULO,TEXTO,NER
0,santanita,fundado 1964 gladys vera / raiza portillo / ot...,"gladys vera,raiza portillo,raiza portillo,glad..."
2,"cardenales del exito, los",fundado 1962 ricardo aguirre / ricardo cepeda ...,"ricardo aguirre,ricardo cepeda,nerio ríos,jesú..."
3,"zagales, los",fundado 1977 daniel méndez / luis germán brice...,"daniel méndez,luis germán briceño,angel fuenma..."
4,maragaita,fundado 1987 astolfo romero / carlos méndez / ...,"astolfo romero,carlos méndez,gladys vera,astol..."
5,barrio obrero,fundado 1955 alberto silva / carmencita silva ...,"alberto silva,carmencita silva,silva,héctor si..."
7,gran coquivacoa,fundado 1968 neguito borjas / osías acosta / e...,"neguito borjas,osías acosta,ender carruyo,punt..."
8,vhg (venezuela habla gaiteando),fundado 1989 danelo badell / renato aguirre / ...,"danelo badell,renato aguirre,heberán añez,rena..."
9,gaiteros de ziruma,fundado 2004 ender rojas / alí ojeda / daniela...,"ender rojas,alí ojeda,daniela palma,gerardo mo..."
10,racoa,fundado 1990 yelitza vílchez / agustín méndez ...,"yelitza vílchez,agustín méndez,javier esis,lul..."
11,gaiteros de pillopo,fundado 1980 astolfo romero / danelo badell / ...,"astolfo romero,danelo badell,eroy chacín,pa qu..."


Y finalmente guardamos los datos en un archivo de Excel, para ser utilizados en la siguiente fase, donde crearemos la red de colaboración usando __NetworkX__ como herramienta principal para la exploración de datos:

In [15]:
nombre_archivo = 'Agrupaciones_NER_flair(v3).xlsx'
agrupaciones_df.to_excel(nombre_archivo)